# Structured Outputs

Structured outputs are a capability that allows language models to format responses in specific, well-defined structures rather than just generating free-form text.
This enables LLMs to deliver answers in formats like **JSON**, **XML**, and others.

**How It Works**:
1. Define a schema that describes the exact format you want
2. The LLM generates content that strictly adheres to this predefined structure
3. You get consistently formatted responses that are easier to parse and use in applications

There are two ways to ask an LLM to format a response in a specific schema:
1. Describe the desired result in your prompt
2. Use structured output capabilities and pass a response schema

Let's start by adding our dependencies, apiKey, and `ChatClient`

In [1]:
%useLatestDescriptors
%use spring-ai-openai
USE { dependencies { implementation("com.fasterxml.jackson.module:jackson-module-kotlin:2.20.1") } }

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.7)
    .build()

val chatClient = ChatClient.create(
    OpenAiChatModel.builder()
        .openAiApi(openAiApi)
        .defaultOptions(openAiOptions)
        .build()
)

First, let's try describing our desired response format in the prompt and see what happens:

In [3]:
val response = chatClient
    .prompt()
    .system("The response must be a valid JSON object.")
    .user(
        """
        What is the firstName and lastName of the person in this sentence?

        "Aurora Skyfield announced her candidacy for the local city council yesterday."
        """
    )
    .call()
    .content()

response

{
  "firstName": "Aurora",
  "lastName": "Skyfield"
}

As we can see, the result is JSON, so we can convert it to a Kotlin class:

In [4]:
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.Json

@Serializable
data class Person(val firstName: String, val lastName: String)

Json.decodeFromString<Person>(response)

Person(firstName=Aurora, lastName=Skyfield)

However, this approach for getting data isn't reliable,
as the prompt doesn't guarantee that the result will be returned exactly in this format.

So let's use a method that will definitely return a structured response —
and directly as a Kotlin class.

First, let's define the class we need:

In [5]:
import com.fasterxml.jackson.annotation.JsonCreator
import com.fasterxml.jackson.annotation.JsonProperty

data class MobileDevice @JsonCreator constructor(
    @JsonProperty("name") val name: String,
    @JsonProperty("price") val price: Double,
    @JsonProperty("category") val category: String,
    @JsonProperty("features") val features: List<String> = emptyList()
)

Now we can simply pass it as the expected result:

In [6]:
import org.springframework.ai.chat.client.entity

chatClient
    .prompt("Tell me about the latest smartphone")
    .call()
    .entity<MobileDevice>()

MobileDevice(name=SuperPhone XZ, price=999.99, category=smartphone, features=[6.7-inch AMOLED display, 128GB internal storage, 12GB RAM, Triple rear camera setup, 5G connectivity, 4000mAh battery, Wireless charging, IP68 water and dust resistance])